# IndoML 2023 Tuturial: Part 2
## At Sesame street

### Plan:

1. We will use Huggingface transformers for finetuning pretrained models.
2. This will allow us to easily swap out models to find the most suitable one.
    * This is achieved using the `AutoModel*` and `AutoTokenizer` classes.
3. It also comes with various new libraries integrated for efficient fine-tuning, 
multi-gpu or distributed training, and more.

### Methods that we will try:

1. DistilBERT
2. mBERT/distilmBERT (multilingual)
3. XLM-RoBERTa (multilingual)
4. DeBERTa (multilingual)

### Efficient Training

1. Linear Probing
2. PEFT
3. Distilled Models

### Libraries to explore

1. 🤗 Transfomers > Trainer
2. 🤗 Evaluate
2. wandb

## Load `dataset`

In [1]:
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

tqdm.pandas()

dataset = load_dataset("AmazonScience/massive")


No config specified, defaulting to: massive/all
Reusing dataset massive (/home/bytestorm/.cache/huggingface/datasets/AmazonScience___massive/all/1.0.0/71d360eb7d7a18565ff8c10609cebf714fce3cc390e173ba5b02ffd48543cdc1)
100%|██████████| 3/3 [00:00<00:00, 294.35it/s]


## Load `AutoTokenizer`

In [2]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


def tokenize_function(example):
    return tokenizer(example["utt"], truncation=True)



## Preprocessing

### Using the Trainer class

We will be using the HF Trainer class to handle a lot of the boilerplate tasks. This Trainer class expects certain format for the dataset.

1. Tokenize the dataset
2. A "labels" column should exits

In [3]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Parameter 'function'=<function tokenize_function at 0x7f98799cb1f0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████| 152/152 [00:16<00:00,  9.42ba/s]


In [4]:
tokenized_datasets['train'][0].keys()

dict_keys(['id', 'locale', 'partition', 'scenario', 'intent', 'utt', 'annot_utt', 'worker_id', 'slot_method', 'judgments', 'input_ids', 'attention_mask'])

In [5]:
# Remove columns that are not needed
to_remove = list(tokenized_datasets['train'][0].keys())

# Don't remove these columns
to_remove.remove('input_ids')
to_remove.remove('attention_mask')
to_remove.remove('intent')

tokenized_datasets = tokenized_datasets.remove_columns(to_remove)
tokenized_datasets = tokenized_datasets.rename_column('intent', 'labels')


In [6]:
tokenized_datasets['train'].features

{'labels': ClassLabel(num_classes=60, names=['datetime_query', 'iot_hue_lightchange', 'transport_ticket', 'takeaway_query', 'qa_stock', 'general_greet', 'recommendation_events', 'music_dislikeness', 'iot_wemo_off', 'cooking_recipe', 'qa_currency', 'transport_traffic', 'general_quirky', 'weather_query', 'audio_volume_up', 'email_addcontact', 'takeaway_order', 'email_querycontact', 'iot_hue_lightup', 'recommendation_locations', 'play_audiobook', 'lists_createoradd', 'news_query', 'alarm_query', 'iot_wemo_on', 'general_joke', 'qa_definition', 'social_query', 'music_settings', 'audio_volume_other', 'calendar_remove', 'iot_hue_lightdim', 'calendar_query', 'email_sendemail', 'iot_cleaning', 'audio_volume_down', 'play_radio', 'cooking_query', 'datetime_convert', 'qa_maths', 'iot_hue_lightoff', 'iot_hue_lighton', 'transport_query', 'music_likeness', 'email_query', 'play_music', 'audio_volume_mute', 'social_post', 'alarm_set', 'qa_factoid', 'calendar_set', 'play_game', 'alarm_remove', 'lists_re

## Dataloaders [Optional]

- Instead of going through individual samples in the dataset, we would like to 
batches of samples to train our model. 
- Dataloader
    1. Creates an "iterator" over the dataset, which returns a batch of samples every turn
    2. Handles shuffling
    3. Collates samples into batches, by padding the input sequences to the maximum length in the batch

In [7]:
# from torch.utils.data import DataLoader

# train_dataloader = DataLoader(
#     tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
# )
# eval_dataloader = DataLoader(
#     tokenized_datasets["validation"], batch_size=8, collate_fn=data_collator
# )

In [8]:
# Check the shape of the tensors
# for batch in tqdm(train_dataloader):
#     break
# {k: v.shape for k, v in batch.items()}

```python
{'labels': torch.Size([8]),
 'input_ids': torch.Size([8, 25]),
 'attention_mask': torch.Size([8, 25])}
```

## Load model using an `AutoModel`

In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=tokenized_datasets['train'].features['labels'].num_classes)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_clas

In [10]:
# Let's check the output of a single forward pass
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>                                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'batch' is not defined

## Setup Evaluation Metric

In [11]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [12]:
# We need to define a compute_metric function that is supported by the Trainer output
# It basically converts the logits to predictions and then calls the metric
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

## Setup Trainer and Finetuning

1. When testing your code for the first time, it is a better idea to use a smaller dataset,
so that you can quickly iterate over your code.
2. We have done the same here by using 128 + 32 samples from the dataset, for training and validation respectively.

In [21]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    max_steps=100,
    # warmup_steps=500,
    # weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    report_to="none" # set "wandb" here to log to wandb
)

PyTorch: setting up devices


**Wandb Logging**

When training ML models, keeping track of various hyperparameters, the training and validation losses, and evaluation metrics is very important.
`wandb` can help you here.

To install wandb, run following commands in your terminal:

```bash
pip install wandb
# Then login to your wandb account
wandb login
```

Once, that is done, in TrainingArguments, set the `report_to` argument to `wandb`.

In [22]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer.train()